In [ ]:
import base64
import json
import os
from dotenv import load_dotenv
from openai import OpenAI
client = OpenAI()

# Load environment variables
load_dotenv()

# Initialize OpenAI client
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

filename = "termsheet-ch1422250584-en.pdf"
with open(f"..\\data\\{filename}", "rb") as f:
    data = f.read()

base64_string = base64.b64encode(data).decode("utf-8")

# Define the function schema
# Load properties only
with open("..\\schemas\\ibt-v2.json", "r", encoding="utf-8") as f:
    properties = json.load(f)

tools = [
    {
        "type": "function",
        "name": "extract_instrument_data",
        "description": "Extract instrument data from document",
        "parameters": {
            "type": "object",
            "properties": properties,
            "required": ["instrument"]
        }
    }
]


In [ ]:
# Send the request
response = client.responses.create(
    model="gpt-4.1",
    input=[
        {
            "role": "user",
            "content": [
                {
                    "type": "input_file",
                    "filename": filename,
                    "file_data": f"data:application/pdf;base64,{base64_string}",
                },
                {
                    "type": "input_text",
                    "text": "Extract instrument data according to the JSON schema from the attached document.",
                },
            ],
        },
    ],
    tools=tools,
)

# Get the first output item
output_item = response.output[0]

# Get the arguments string
arguments_str = output_item.arguments

# Parse the JSON string into Python dict
data = json.loads(arguments_str)

# Pretty print
print(json.dumps(data, indent=2))

# Split filename and extension
name, ext = os.path.splitext(filename)

# Build new filename with .json extension
json_filename = f"{name}.json"

# Save to file
with open(f"..\\data\\{json_filename}", "w", encoding="utf-8") as f:
    json.dump(data, f, indent=2, ensure_ascii=False)

JSONDecodeError: Expecting ',' delimiter: line 59031 column 1 (char 298497)